In [1]:
from datetime import datetime, date, timedelta
import calendar
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

from bs4 import BeautifulSoup

import requests
import pandas as pd
import numpy as np
from statistics import mode
import plotly.express as px
import plotly.graph_objects as go
import time

import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
import plotly.express as px
import plotly.io as pio

import boto3
from sqlalchemy import create_engine, text

In [2]:
cities = ['Mont Saint Michel', 'St Malo', "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg", "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon", "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille", "Aix en Provence", "Avignon", "Uzes", "Nimes", "Aigues Mortes", "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"]

In [3]:
url = "https://nominatim.openstreetmap.org/search?"
user_agent = "project_kayak/1.0 (mouton.cl@gmail.com)"

def find_gps(address, format='json'):
    params = {"q": address, "format": format}
    headers = {"User-Agent": user_agent}
    response = requests.get(url, params=params, headers=headers)
    
    if response.status_code != 200:
        print(f"Erreur: Réponse avec le code de statut {response.status_code} pour l'adresse '{address}'")
        return None
    
    try:
        return response.json()
    except ValueError:
        print(f"Erreur: Impossible de parser la réponse JSON pour l'adresse '{address}'")
        print(f"Contenu de la réponse: {response.text}")
        return None

columns = ['id', 'name', 'latitude', 'longitude']
data = []

for count, city in enumerate(cities):
    response = find_gps(address = city)
    row =[count, city, response[0]['lat'], response[0]['lon']]
    data.append(row)
    time.sleep(1)
    
df_cities = pd.DataFrame(data=data, columns=columns)

In [6]:
display(df_cities.head())
df_cities.to_csv('files/cities_infos.csv', index=False)

,id,name,latitude,longitude
0,0,Mont Saint Michel,48.6359541,-1.511459954959514
1,1,St Malo,49.314695,-96.9538228
2,2,Bayeux,49.2764624,-0.7024738
3,3,Le Havre,49.4938975,0.1079732
4,4,Rouen,49.4404591,1.0939658


# Weather

In [8]:
api_key = "01c1f72fc7a91c71b390a1cd59948cd0"

In [9]:
# Extraire les coordonnées de la première ville
lat = df_cities.iloc[0]['latitude']
lon = df_cities.iloc[0]['longitude']

# Requête API pour la première ville
response = requests.get(f"https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&units=metric&appid={api_key}")

# Vérifier si la requête a réussi
if response.status_code == 200:
    data = response.json()  # Convertir la réponse en format JSON
    # Maintenant, vous pouvez traiter les données comme vous le souhaitez
    print(data)
else:
    print("La requête a échoué, code d'erreur :", response.status_code)


{'lat': 48.636, 'lon': -1.5115, 'timezone': 'Europe/Paris', 'timezone_offset': 7200, 'current': {'dt': 1721227982, 'sunrise': 1721190113, 'sunset': 1721246529, 'temp': 24.58, 'feels_like': 24.71, 'pressure': 1020, 'humidity': 62, 'dew_point': 16.82, 'uvi': 3.78, 'clouds': 74, 'visibility': 10000, 'wind_speed': 1.22, 'wind_deg': 151, 'wind_gust': 3.05, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}]}, 'minutely': [{'dt': 1721228040, 'precipitation': 0}, {'dt': 1721228100, 'precipitation': 0}, {'dt': 1721228160, 'precipitation': 0}, {'dt': 1721228220, 'precipitation': 0}, {'dt': 1721228280, 'precipitation': 0}, {'dt': 1721228340, 'precipitation': 0}, {'dt': 1721228400, 'precipitation': 0}, {'dt': 1721228460, 'precipitation': 0}, {'dt': 1721228520, 'precipitation': 0}, {'dt': 1721228580, 'precipitation': 0}, {'dt': 1721228640, 'precipitation': 0}, {'dt': 1721228700, 'precipitation': 0}, {'dt': 1721228760, 'precipitation': 0}, {'dt': 1721228820, 'p

In [10]:
def get_weather(latitude, longitude):
    # We set units = metric to have Celsius temperature instead of Kelvin
    # We only keep information about daily forecast for 8 days 
    url = f"https://api.openweathermap.org/data/3.0/onecall?lat={latitude}&lon={longitude}&units=metric&exclude=current,minutely,hourly,alerts&appid={api_key}"
    response = requests.get(url)
    data = response.json()

    if data :
        humidity = data["daily"][0]["humidity"]                     # % of humidity
        temp_avg = data["daily"][0]["temp"]["day"]                  # average temperature
        temp_min = data["daily"][0]["temp"]["min"]                  # minimum temperature
        temp_max = data["daily"][0]["temp"]["max"]                  # maximum temperature
        temp_perceived = data["daily"][0]["feels_like"]["day"]      # perceived temperature
        wind_speed = data["daily"][0]["wind_speed"]                 # wind speed in meter/seconde
        clouds = data["daily"][0]["clouds"]                         # % of cloudiness
        pop = data["daily"][0]["pop"]                               # probability of precipitation, between 0 and 1
        
        return humidity, temp_avg, temp_min, temp_max, temp_perceived, wind_speed, clouds, pop
    else:
        return None

In [13]:
weather_results = []

for index, row in df_cities.iterrows():
    name = row["name"]
    latitude = row["latitude"]
    longitude = row["longitude"]

    humidity, temp_avg, temp_min, temp_max, temp_perceived, wind_speed, clouds, pop = get_weather(latitude, longitude)

    weather_results.append({"name": name, 
                            "latitude": latitude, 
                            "longitude": longitude, 
                            "Humidity": humidity, 
                            "Temperature_avg": temp_avg,
                            "Temperature_min": temp_min,
                            "Temperature_max": temp_max,
                            "Temperature_perceived": temp_perceived,
                            "Wind_speed": wind_speed,
                            "Cloudiness": clouds,
                            "Precipitation_prob" : round(pop*100) # to have the probability of precipitation in %
                            })

df = pd.DataFrame(weather_results)
df.head()

,name,latitude,longitude,Humidity,Temperature_avg,Temperature_min,Temperature_max,Temperature_perceived,Wind_speed,Cloudiness,Precipitation_prob
0,Mont Saint Michel,48.6359541,-1.511459954959514,63,22.64,13.17,24.58,22.60,2.83,57,0
1,St Malo,49.314695,-96.9538228,66,18.05,8.97,20.33,17.63,3.19,7,0
2,Bayeux,49.2764624,-0.7024738,66,21.01,12.61,22.14,20.89,3.98,71,0
3,Le Havre,49.4938975,0.1079732,73,19.09,16.27,20.51,18.96,6.36,19,0
4,Rouen,49.4404591,1.0939658,50,23.75,12.66,24.48,23.49,3.32,19,0


In [14]:
# Add index column to the dataframe
df = df.reset_index()
df.head()

,index,name,latitude,longitude,Humidity,Temperature_avg,Temperature_min,Temperature_max,Temperature_perceived,Wind_speed,Cloudiness,Precipitation_prob
0,0,Mont Saint Michel,48.6359541,-1.511459954959514,63,22.64,13.17,24.58,22.60,2.83,57,0
1,1,St Malo,49.314695,-96.9538228,66,18.05,8.97,20.33,17.63,3.19,7,0
2,2,Bayeux,49.2764624,-0.7024738,66,21.01,12.61,22.14,20.89,3.98,71,0
3,3,Le Havre,49.4938975,0.1079732,73,19.09,16.27,20.51,18.96,6.36,19,0
4,4,Rouen,49.4404591,1.0939658,50,23.75,12.66,24.48,23.49,3.32,19,0


In [15]:
# Save the dataframe in a csv file
df.to_csv("files/cities_forecast_weather.csv", index=False)

In [16]:
# Determine what are the best 5 cities destination
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, 
                    cols=3, 
                    start_cell="bottom-left",
                    specs=[[{"type": "domain"}, {"type": "domain"},{"type": "domain"}], 
                    [{"type": "domain"},{"type": "domain"},{"type": "domain"}]],
                    subplot_titles=("Top 5 cities with the least humidity (%)", 
                                    "Top 5 cities with the best day temperature(°C)", 
                                    "Top 5 cities with the least speed wind (m/s)",
                                    "Top 5 cities with the least cloudiness (%)",
                                    "Top 5 cities with the least probability of rain (%)"))

fig.add_trace(go.Pie(
    labels=df.sort_values(by=['Humidity']).head(5)["name"],
    values=df.sort_values(by=['Humidity']).head(5)["Humidity"],
    textinfo="label+value"),
    row=1, col=1
)

fig.add_trace(go.Pie(
    labels=df.sort_values(by=['Temperature_avg'], ascending=False).head(5)["name"],
    values=df.sort_values(by=['Temperature_avg'], ascending=False).head(5)["Temperature_avg"],
    textinfo="label+value"),
    row=1, col=2
)

fig.add_trace(go.Pie(
    labels=df.sort_values(by=['Wind_speed']).head(5)["name"],
    values=df.sort_values(by=['Wind_speed']).head(5)["Wind_speed"],
    textinfo="label+value"),
    row=1, col=3
)

# Ajouter une petite valeur à toutes les valeurs de la catégorie "Cloudiness"
df['Cloudiness_adjusted'] = df['Cloudiness'] + 0.1

fig.add_trace(go.Pie(
    labels=df.sort_values(by=['Cloudiness_adjusted']).head(5)["name"],
    values=df.sort_values(by=['Cloudiness_adjusted']).head(5)["Cloudiness_adjusted"],
    textinfo="label+value"),
    row=2, col=1
)

# Ajouter une petite valeur à toutes les valeurs de la catégorie "Precipitation_prob"
df['Precipitation_prob_adjusted'] = df['Precipitation_prob'] + 0.1

fig.add_trace(go.Pie(
    labels=df.sort_values(by=['Precipitation_prob_adjusted']).head(5)["name"],
    values=df.sort_values(by=['Precipitation_prob_adjusted']).head(5)["Precipitation_prob_adjusted"],
    textinfo="label+value"),
    row=2, col=2
)

fig.update_layout(
    # title='Top 5 cities with the least humidity',
    height=700, 
    width=1400,
    showlegend=False
)

fig.show()

In [17]:
# Tri par ordre croissant de la catégorie "Cloudiness"
df_cities_sorted = df.sort_values(by='Temperature_avg', ascending=True)

# Affichage des premières lignes du DataFrame trié
df_cities_sorted.head(10)


,index,name,latitude,longitude,Humidity,Temperature_avg,Temperature_min,Temperature_max,Temperature_perceived,Wind_speed,Cloudiness,Precipitation_prob,Cloudiness_adjusted,Precipitation_prob_adjusted
1,1,St Malo,49.314695,-96.9538228,66,18.05,8.97,20.33,17.63,3.19,7,0,7.1,0.1
3,3,Le Havre,49.4938975,0.1079732,73,19.09,16.27,20.51,18.96,6.36,19,0,19.1,0.1
2,2,Bayeux,49.2764624,-0.7024738,66,21.01,12.61,22.14,20.89,3.98,71,0,71.1,0.1
9,9,Chateau du Haut Koenigsbourg,48.249410749999996,7.344320233724503,58,21.02,11.63,22.14,20.69,2.28,37,39,37.1,39.1
34,34,La Rochelle,46.1591126,-1.1520434,64,21.34,14.96,23.07,21.20,3.46,5,0,5.1,0.1
6,6,Amiens,49.8941708,2.2956951,62,22.40,13.04,23.44,22.31,4.81,39,63,39.1,63.1
13,13,Dijon,47.3215806,5.0414701,61,22.52,13.91,25.80,22.42,3.05,23,21,23.1,21.1
0,0,Mont Saint Michel,48.6359541,-1.511459954959514,63,22.64,13.17,24.58,22.60,2.83,57,0,57.1,0.1
11,11,Eguisheim,48.0447968,7.3079618,60,22.86,13.98,25.00,22.77,2.47,36,5,36.1,5.1
7,7,Lille,50.6365654,3.0635282,56,23.02,14.08,24.22,22.84,3.79,39,100,39.1,100.1


In [18]:
top5_city=df_cities_sorted[0:5]

In [19]:
top5_city

,index,name,latitude,longitude,Humidity,Temperature_avg,Temperature_min,Temperature_max,Temperature_perceived,Wind_speed,Cloudiness,Precipitation_prob,Cloudiness_adjusted,Precipitation_prob_adjusted
1,1,St Malo,49.314695,-96.9538228,66,18.05,8.97,20.33,17.63,3.19,7,0,7.1,0.1
3,3,Le Havre,49.4938975,0.1079732,73,19.09,16.27,20.51,18.96,6.36,19,0,19.1,0.1
2,2,Bayeux,49.2764624,-0.7024738,66,21.01,12.61,22.14,20.89,3.98,71,0,71.1,0.1
9,9,Chateau du Haut Koenigsbourg,48.249410749999996,7.344320233724503,58,21.02,11.63,22.14,20.69,2.28,37,39,37.1,39.1
34,34,La Rochelle,46.1591126,-1.1520434,64,21.34,14.96,23.07,21.20,3.46,5,0,5.1,0.1


In [20]:
# Save the dataframe in a csv file
top5_city.to_csv("files/top_5city.csv")

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Lire les données météo
weather_data = pd.read_csv('files/cities_forecast_weather.csv')

# Vérifier les données de weather_data
print("Données météo lues depuis le fichier :")
print(weather_data.head())

# Initialiser l'agent utilisateur pour les requêtes
agent = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

# Construire les URL de recherche pour chaque ville du dataset météo
search_urls = ['https://www.booking.com/searchresults.fr.html?ss='+city+'&nflt=review_score%3D60' for city in weather_data['name']]

# Récupérer les URL des hôtels (25 premiers sur la page de recherche de Booking) pour chaque URL de recherche
hotels_data = []

for url in search_urls:
    page = requests.get(url, headers=agent)
    soup = BeautifulSoup(page.content, "html.parser")
    
    hotel_cards = soup.find_all('div', class_='sr_item')
    
    for card in hotel_cards:
        hotel_data = {}
        
        # Nom de l'hôtel
        hotel_data['name'] = card.find('span', class_='sr-hotel__name').text.strip()
        
        # Adresse de l'hôtel
        address_element = card.find('div', class_='address')
        hotel_data['address'] = address_element.text.strip() if address_element else 'N/A'
        
        # Description de l'hôtel
        description_element = card.find('div', class_='hotel_desc')
        hotel_data['description'] = description_element.text.strip() if description_element else 'N/A'
        
        # Score de l'hôtel
        score_element = card.find('div', class_='bui-review-score__badge')
        hotel_data['score'] = score_element.text.strip() if score_element else 'N/A'
        
        # Bonus ou équipements importants
        bonus_element = card.find('div', class_='facility-badge')
        hotel_data['bonus'] = bonus_element.text.strip() if bonus_element else 'N/A'
        
        # URL de l'hôtel
        url_element = card.find('a', class_='hotel_name_link url')
        hotel_data['url'] = url_element['href'] if url_element else 'N/A'
        
        hotels_data.append(hotel_data)

# Vérifier les données des hôtels extraites
print("\nExemples de données d'hôtels extraites :")
print(hotels_data[:5])

# Créer un DataFrame à partir des données des hôtels
if hotels_data:
    booking_data = pd.DataFrame(hotels_data)
    
    # Ajouter les colonnes supplémentaires de weather_data à booking_data
    booking_data = pd.concat([weather_data] * (len(booking_data) // len(weather_data)), ignore_index=True)
    booking_data = booking_data.drop(['latitude', 'longitude', 'Humidity', 'Temperature_avg', 'Temperature_min', 'Temperature_max', 'Temperature_perceived', 'Wind_speed', 'Cloudiness', 'Precipitation_prob'], axis=1)
    
    # Nettoyer les données textuelles
    for col in ['description', 'score', 'bonus']:
        booking_data[col] = booking_data[col].str.replace(r'\n', ' ').str.replace(r'\xa0', ' ')
    
    # Extraire la latitude et la longitude à partir des coordonnées et supprimer la colonne 'latlon'
    booking_data[['latitude', 'longitude']] = booking_data['latlon'].str.split(',', expand=True)
    booking_data = booking_data.drop('latlon', axis=1)
    
    # Convertir les types de données
    booking_data[['latitude', 'longitude']] = booking_data[['latitude', 'longitude']].astype(float)
    
    # Utiliser une expression régulière pour extraire le score
    booking_data['score'] = booking_data['score'].str.extract(r'(\d+[\.,]?\d*)', expand=False).str.replace(',', '.').astype(float)
    
    print("\nExemple de données finales après traitement :")
    print(booking_data.head())
else:
    print("Erreur : Aucune donnée d'hôtel n'a été extraite.")



In [30]:
# On sauvegarde les données sur les differents hotels

booking_data.to_csv('files/booking_data.csv', index=False)

In [ ]:
booking_data.sample(5)

# STOCKAGE AWS S3

In [46]:
# On cree un bucket s3 Amazon

ACCESS_KEY_ID = os.environ.get('aws_access_key_id')
SECRET_ACCESS_KEY = os.environ.get('aws_secret_access_key')

bucketname = 'booking-mouton-files'

In [ ]:
session = boto3.Session(
    aws_access_key_id= ACCESS_KEY_ID,
    aws_secret_access_key= SECRET_ACCESS_KEY
)

s3 = session.client("s3")

bucket = s3.create_bucket(Bucket=bucketname, ACL='public-read-write', CreateBucketConfiguration={'LocationConstraint': 'eu-west-3'})

In [ ]:

# Chargement des fichiers

s3.Bucket(bucketname).upload_file('files/booking_data.csv','booking_data.csv')
s3.Bucket(bucketname).upload_file('files/cities_forecast_weather.csv','cities_forecast_weather.csv')

In [ ]:
# Telechargement d'un dataset pour tester le upload

booking_data = pd.read_csv(f"https://{bucketname}.s3.eu-west-3.amazonaws.com/booking_data.csv")
booking_data.head()

In [ ]:
login = open("login.txt", "r").readlines()

In [ ]:
# Configuration de RDS

filename = "booking_data.csv"

s3.download_file(bucketname, filename, filename)

data = pd.read_csv(filename)

# Connexion à la base de données MySQL

# engine = create_engine(f"mysql+pymysql://{DBUSER}:{DBPASS}@{DBHOST}:{PORT}/{DBNAME}")
engine = create_engine(f"mysql+pymysql://{login[0].strip()}:{login[1].strip()}@{login[2].strip()}:{login[3].strip()}/{login[4].strip()}")

# Stocker les données dans la base de données
data.to_sql('booking_data', con=engine, index=False, if_exists='replace')

os.remove(filename)

In [ ]:
engine

In [ ]:
weather_data = pd.read_csv('files/cities_weather_data.csv')

In [ ]:
# stocker les données météo et de réservation sous forme de tables SQL

weather_data.to_sql('weather', con=engine, if_exists='replace', index=False)
booking_data.to_sql('booking', con=engine, if_exists='replace', index=False)

In [ ]:
# Obtenir des données météorologiques à partir d'une table SQL

weather_query = text("SELECT * FROM weather WHERE quality > 10 ORDER BY quality DESC LIMIT 5")
weather_selection = pd.read_sql(weather_query, engine)
weather_selection.head()

In [ ]:
# choisir les hôtels dont nous avons besoin en fonction de la météo

hotel_query = text("SELECT * FROM booking WHERE cities in ('Lyon','Paris','Annecy','La Rochelle','Avignon')")
hotel_selection = pd.read_sql(hotel_query, engine)
hotel_selection.head()